# Notes - Xarray

## Terminology

- DataArray: A multi-dimensional array with labeled or named dimensions. DataArray objects add metadata such as dimension names, coordinates, and attributes. For example, an array is var(time, level, lat, lon).
- DataSet: A dict-like collection of DataArray objects with aligned dimensions. For example, a dataset contains temperature(time, level, lat, lon) and precipitation(time, lat, lon).

## References

- Unidata Xarray Introduction, https://unidata.github.io/python-training/workshop/XArray/xarray-introduction/
- Xarray quick overview, https://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html
- Xarray computation, https://docs.xarray.dev/en/stable/user-guide/computation.html


In [1]:
import numpy as np
import xarray as xr
import io, os, sys, types

## Create a DataArray

Xarray - https://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html#create-a-dataarray

Unidata - https://unidata.github.io/python-training/workshop/XArray/xarray-introduction/#DataArray

In [2]:
#--- Create some sample "temperature" data
data = 283 + 5 * np.random.randn(5, 3, 4)

time = np.arange(0,5)
lat = np.linspace(-120., 60., 3)
lon = np.linspace(25.,55.,4)

#--- create a DataArray & set attributes
temp = xr.DataArray(data, dims=['time', 'lat', 'lon'], coords=[time, lat, lon])

temp.attrs['units'] = "K"
temp.attrs['long_name'] = "Temperature"

with xr.set_options(keep_attrs=True):  # keep attributes after operation
  temp_degC = temp - 273.15
temp_degC.attrs['units'] = "C"
temp_degC

#--- put all in one line
arr = xr.DataArray(np.random.RandomState(0).randn(2, 3), [("x", ["a", "b"]), ("y", [10, 20, 30])])
arr

<xarray.DataArray (x: 2, y: 3)>
array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30

## Selection

Unidata - https://unidata.github.io/python-training/workshop/XArray/xarray-introduction/#Selection

### Selection Method 1: use indexing

In [3]:
#--- Method 1: use indexing
var = temp[0, 1:2, :]
var

<xarray.DataArray (lat: 1, lon: 4)>
array([[276.2333842 , 283.40064275, 288.13928095, 272.95644252]])
Coordinates:
    time     int64 0
  * lat      (lat) float64 -30.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_name:  Temperature

### Selection Method 2: Use name dimension & slicing

In [4]:
#--- Method 2; use name dimension
temp.coords  # check out variable dimension
print(temp.coords)

#--- select specific values in coordinates
var = temp.sel(time=1, lat=-30., lon=25)
print('------------')
print(var)

var = temp.sel(time=1, lon=25)
print('------------')
print(var)

#--- Slicing with Selection
var = temp.sel(time=slice(0,2), lat=-30., lon=slice(-1000.,1000.))
print('------------')
print(var)

Coordinates:
  * time     (time) int64 0 1 2 3 4
  * lat      (lat) float64 -120.0 -30.0 60.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
------------
<xarray.DataArray ()>
array(270.01627945)
Coordinates:
    time     int64 1
    lat      float64 -30.0
    lon      float64 25.0
Attributes:
    units:      K
    long_name:  Temperature
------------
<xarray.DataArray (lat: 3)>
array([282.58627381, 270.01627945, 282.59929309])
Coordinates:
    time     int64 1
  * lat      (lat) float64 -120.0 -30.0 60.0
    lon      float64 25.0
Attributes:
    units:      K
    long_name:  Temperature
------------
<xarray.DataArray (time: 3, lon: 4)>
array([[276.2333842 , 283.40064275, 288.13928095, 272.95644252],
       [270.01627945, 274.42040253, 280.97599144, 283.07655771],
       [276.92901922, 278.1175962 , 284.07623506, 279.15319978]])
Coordinates:
  * time     (time) int64 0 1 2
    lat      float64 -30.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_n

### Selection Method 3: use .loc

In [5]:
#*** Useful if already knowing the range to each coordinate

# temp is temp(time, lat, lon)
var = temp.loc[0:4, -120:30, :]
print(var)

<xarray.DataArray (time: 5, lat: 2, lon: 4)>
array([[[274.56742377, 286.19145406, 282.79127358, 279.61738955],
        [276.2333842 , 283.40064275, 288.13928095, 272.95644252]],

       [[282.58627381, 282.33892942, 285.58045534, 285.97582845],
        [270.01627945, 274.42040253, 280.97599144, 283.07655771]],

       [[281.55582561, 279.83247111, 275.14437937, 278.1872848 ],
        [276.92901922, 278.1175962 , 284.07623506, 279.15319978]],

       [[280.59120529, 281.86526717, 285.53500421, 283.65086406],
        [277.51106314, 279.68549686, 282.54855105, 278.98938536]],

       [[280.09251015, 288.58874972, 281.98018046, 286.40848067],
        [286.39284547, 290.23051669, 287.96227817, 284.40292077]]])
Coordinates:
  * time     (time) int64 0 1 2 3 4
  * lat      (lat) float64 -120.0 -30.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_name:  Temperature


### Selection Method 4: where() to conditionally switch between values

In [6]:
#--- use where()
var = temp.where(temp > 280.)  # if temp < 280., it would become nan
var

#--- create a mask, but fail.
#xr.where(temp > 280., "positive", "negative")

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[         nan, 286.19145406, 282.79127358,          nan],
        [         nan, 283.40064275, 288.13928095,          nan],
        [280.55046962, 283.37627498, 288.8236457 , 285.3985922 ]],

       [[282.58627381, 282.33892942, 285.58045534, 285.97582845],
        [         nan,          nan, 280.97599144, 283.07655771],
        [282.59929309, 282.94333592, 290.13876543, 282.15509721]],

       [[281.55582561,          nan,          nan,          nan],
        [         nan,          nan, 284.07623506,          nan],
        [280.36732637,          nan, 283.63861777, 285.54944245]],

       [[280.59120529, 281.86526717, 285.53500421, 283.65086406],
        [         nan,          nan, 282.54855105,          nan],
        [292.64266012, 288.09948101, 287.89699023, 280.25950925]],

       [[280.09251015, 288.58874972, 281.98018046, 286.40848067],
        [286.39284547, 290.23051669, 287.96227817, 284.40292077],
        [285.39813554, 285.02720413, 284.47539365, 285.70626429]]])
Coordinates:
  * time     (time) int64 0 1 2 3 4
  * lat      (lat) float64 -120.0 -30.0 60.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_name:  Temperature

## Computation &

### Basic numerical operation

In [7]:
#--- arithmetic operation
xr.set_options(keep_attrs =True)  # keep attributes after operations
temp_degC = temp - 273.15
temp_degC.attrs['units']="C"
temp_degC

#--- mean and others
#    a list: mean, min, max, std, sum, weighted
temp_tavg = temp.mean("time")
temp_tavg

temp_tiavg = temp.mean(["time","lon"])
temp_tiavg

<xarray.DataArray (lat: 3)>
array([282.15406253, 280.76090447, 284.7159507 ])
Coordinates:
  * lat      (lat) float64 -120.0 -30.0 60.0
Attributes:
    units:      K
    long_name:  Temperature

### Broadcasting by dimension name

https://docs.xarray.dev/en/stable/user-guide/computation.html#broadcasting-by-dimension-name

In [8]:
a = xr.DataArray([1, 2], [("x", ["a", "b"])])
b = xr.DataArray([-1, -2, -3], [("y", [10, 20, 30])])
c = xr.DataArray(np.arange(6).reshape(3, 2), [b["y"], a["x"]])

#print(a)
#print(b)
#print(c)

#--- With xarray, and their dimensions are expanded automatically:
print(a*b)  # 2x3 array
print(b*a)  # 3x2 array

#--- explicitly broadcast xarray data structures by using the broadcast()
a2, b2 = xr.broadcast(a, b)
print(a2)
print(b2)


<xarray.DataArray (x: 2, y: 3)>
array([[-1, -2, -3],
       [-2, -4, -6]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30
<xarray.DataArray (y: 3, x: 2)>
array([[-1, -2],
       [-2, -4],
       [-3, -6]])
Coordinates:
  * y        (y) int64 10 20 30
  * x        (x) <U1 'a' 'b'
<xarray.DataArray (x: 2, y: 3)>
array([[1, 1, 1],
       [2, 2, 2]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30
<xarray.DataArray (x: 2, y: 3)>
array([[-1, -2, -3],
       [-1, -2, -3]])
Coordinates:
  * y        (y) int64 10 20 30
  * x        (x) <U1 'a' 'b'


### Automatic alignment
https://docs.xarray.dev/en/stable/user-guide/computation.html#automatic-alignment



In [9]:
arr = xr.DataArray(np.arange(3), [("x", range(3))])

#--- only operate available elements
print(arr)
print(arr[:-1])
print(arr + arr[:-1])

<xarray.DataArray (x: 3)>
array([0, 1, 2])
Coordinates:
  * x        (x) int64 0 1 2
<xarray.DataArray (x: 2)>
array([0, 1])
Coordinates:
  * x        (x) int64 0 1
<xarray.DataArray (x: 2)>
array([0, 2])
Coordinates:
  * x        (x) int64 0 1


## Plotting